In [11]:
import math
import requests # HTTP request
#import xlswriter
import numpy as np
import pandas as pd

In [12]:
stocks = pd.read_csv ('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [13]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
print (IEX_CLOUD_API_TOKEN)

Tpk_059b97af715d417d9f49f50b51b1c448


In [14]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"
data = requests.get (api_url).json()
print (type(data)) # <-- Response Object
print (data) # <Response [200]> / <Response [404]>

<class 'dict'>
{'avgTotalVolume': 83453054, 'calculationPrice': 'tops', 'change': -0.18, 'changePercent': -0.00146, 'close': 0, 'closeSource': 'flfoacii', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'tlmiEepXa r Ie crei', 'highTime': 1754905722240, 'iexAskPrice': 132.65, 'iexAskSize': 201, 'iexBidPrice': 129.79, 'iexBidSize': 101, 'iexClose': 133.26, 'iexCloseTime': 1716032386768, 'iexLastUpdated': 1709183200048, 'iexMarketPercent': None, 'iexOpen': 130.81, 'iexOpenTime': 1699567944034, 'iexRealtimePrice': 134.58, 'iexRealtimeSize': 310, 'iexVolume': 159346, 'lastTradeTime': 1708642374758, 'latestPrice': 133.86, 'latestSource': 'IEX real time price', 'latestTime': '9:36:16 AM', 'latestUpdate': 1748442619243, 'latestVolume': None, 'low': 0, 'lowSource': 'ai IXmrlieecE pet r', 'lowTime': 173

In [16]:
print (data['avgTotalVolume'])

83453054
